<a href="https://colab.research.google.com/github/ayz1070/CapstoneStockModelingProject/blob/main/LSTM_%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9(%EC%88%98%EC%A0%95v3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

In [9]:
# 데이터 - CSV 파일 가져오기

csv_df = pd.read_csv('005380_현대차.csv')  # CSV 파일명 입력
csv_df

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,금리,환율
0,0,2015-01-02,169000,170500,167000,169000,458014,0.000000,181909,5.38,0.93,31441,1.15,1950,2.0,1093.599976
1,1,2015-01-05,171000,171000,166000,168000,454073,-0.005917,181909,5.34,0.92,31441,1.16,1950,2.0,1111.000000
2,2,2015-01-06,166500,167000,164000,164500,635059,-0.020833,181909,5.23,0.90,31441,1.19,1950,2.0,1108.500000
3,3,2015-01-07,163000,171500,163000,170000,511006,0.033435,181909,5.41,0.93,31441,1.15,1950,2.0,1097.300049
4,4,2015-01-08,172500,178000,171000,178000,902742,0.047059,181909,5.66,0.98,31441,1.10,1950,2.0,1097.800049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,2114,2023-08-02,197800,198800,192400,192500,917045,-0.034604,407147,6.75,0.47,28521,3.64,7000,3.5,1289.050049
2115,2115,2023-08-03,192500,194300,190800,191700,531818,-0.004156,407147,6.72,0.47,28521,3.65,7000,3.5,1295.369995
2116,2116,2023-08-04,191600,193400,190500,192200,378998,0.002608,407147,6.74,0.47,28521,3.64,7000,3.5,1296.400024
2117,2117,2023-08-07,191000,191100,187300,188000,842043,-0.021852,407147,6.59,0.46,28521,3.72,7000,3.5,1303.050049


In [10]:
csv_df = csv_df.loc[:, ['Date', 'Close', 'BPS', 'EPS', '금리', '환율']]
csv_df = csv_df.set_index('Date') # 날짜를 index로 바꿈
csv_df

,Close,BPS,EPS,금리,환율
Date,,,,,
2015-01-02,169000,181909,31441,2.0,1093.599976
2015-01-05,168000,181909,31441,2.0,1111.000000
2015-01-06,164500,181909,31441,2.0,1108.500000
2015-01-07,170000,181909,31441,2.0,1097.300049
2015-01-08,178000,181909,31441,2.0,1097.800049
...,...,...,...,...,...
2023-08-02,192500,407147,28521,3.5,1289.050049
2023-08-03,191700,407147,28521,3.5,1295.369995
2023-08-04,192200,407147,28521,3.5,1296.400024


In [11]:
csv_df.dropna() # 결측치 처리

,Close,BPS,EPS,금리,환율
Date,,,,,
2015-01-02,169000,181909,31441,2.0,1093.599976
2015-01-05,168000,181909,31441,2.0,1111.000000
2015-01-06,164500,181909,31441,2.0,1108.500000
2015-01-07,170000,181909,31441,2.0,1097.300049
2015-01-08,178000,181909,31441,2.0,1097.800049
...,...,...,...,...,...
2023-08-02,192500,407147,28521,3.5,1289.050049
2023-08-03,191700,407147,28521,3.5,1295.369995
2023-08-04,192200,407147,28521,3.5,1296.400024


In [12]:
# 날짜 빼고 정규화 작업 진행 -> MinMaxScaler 사용
scaler = MinMaxScaler()

# 정규화 수행 -> 정규화된 데이터프레임은 scaled_df
scale_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
scaled_df = scaler.fit_transform(csv_df[scale_cols]) # 정규화된 데이터는 넘파이 형태
scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) # Pandas DataFrame 형태로 변경

# 테스트
scaled_df

,Close,BPS,EPS,금리,환율
0,0.511409,0.0,1.000000,0.5,0.102171
1,0.506448,0.0,1.000000,0.5,0.146760
2,0.489087,0.0,1.000000,0.5,0.140353
3,0.516369,0.0,1.000000,0.5,0.111652
4,0.556052,0.0,1.000000,0.5,0.112934
...,...,...,...,...,...
2114,0.627976,1.0,0.887636,1.0,0.603029
2115,0.624008,1.0,0.887636,1.0,0.619225
2116,0.626488,1.0,0.887636,1.0,0.621864
2117,0.605655,1.0,0.887636,1.0,0.638905


In [13]:
# 날짜를 뺐으니 다시 날짜를 index 로 붙여줍니다
scaled_csv_df = scaled_df.set_index(csv_df.index)
scaled_csv_df

,Close,BPS,EPS,금리,환율
Date,,,,,
2015-01-02,0.511409,0.0,1.000000,0.5,0.102171
2015-01-05,0.506448,0.0,1.000000,0.5,0.146760
2015-01-06,0.489087,0.0,1.000000,0.5,0.140353
2015-01-07,0.516369,0.0,1.000000,0.5,0.111652
2015-01-08,0.556052,0.0,1.000000,0.5,0.112934
...,...,...,...,...,...
2023-08-02,0.627976,1.0,0.887636,1.0,0.603029
2023-08-03,0.624008,1.0,0.887636,1.0,0.619225
2023-08-04,0.626488,1.0,0.887636,1.0,0.621864


In [14]:
# 입력데이터, 정답(예측하는)데이터를 정의
# 입력데이터 -> 이전날까지의 종가와 변수들
# 정답데이터 -> 다음날의 종가

feature_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
label_cols = ['Close']

# 입력데이터, 정답데이터 프레임 -> feature_df, label_df
feature_df = scaled_csv_df[feature_cols]
label_df = scaled_csv_df[label_cols]

print(feature_df)
print(label_df)


# DataFrame을 Numpy 형태로 저장
feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

               Close  BPS       EPS   금리        환율
Date                                              
2015-01-02  0.511409  0.0  1.000000  0.5  0.102171
2015-01-05  0.506448  0.0  1.000000  0.5  0.146760
2015-01-06  0.489087  0.0  1.000000  0.5  0.140353
2015-01-07  0.516369  0.0  1.000000  0.5  0.111652
2015-01-08  0.556052  0.0  1.000000  0.5  0.112934
...              ...  ...       ...  ...       ...
2023-08-02  0.627976  1.0  0.887636  1.0  0.603029
2023-08-03  0.624008  1.0  0.887636  1.0  0.619225
2023-08-04  0.626488  1.0  0.887636  1.0  0.621864
2023-08-07  0.605655  1.0  0.887636  1.0  0.638905
2023-08-08  0.607639  1.0  0.887636  1.0  0.650950

[2119 rows x 5 columns]
               Close
Date                
2015-01-02  0.511409
2015-01-05  0.506448
2015-01-06  0.489087
2015-01-07  0.516369
2015-01-08  0.556052
...              ...
2023-08-02  0.627976
2023-08-03  0.624008
2023-08-04  0.626488
2023-08-07  0.605655
2023-08-08  0.607639

[2119 rows x 1 columns]


In [15]:
# sequence_dataset 만들기
def make_sequence_dataset(feature, label, window_size):

  feature_list = []   # 생성될 feature list
  label_list = []     # 생성될 label list

  for i in range(len(feature)-window_size): # range는 전체값에서 window_size를 뺀 값

    feature_list.append(feature[i:i+window_size]) # feature list 에 i번째서 부터 window size 만큼의 입력데이터를 추가
    label_list.append(label[i+window_size]) # label list 에 그 다음 번째('window_size + 1' 번째)의 정답데이터를 추가

  return np.array(feature_list), np.array(label_list)



---



### 5일치 예측 -> **출력층** units수 = 5

In [16]:
window_size = 80  # window_size 만큼의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측

X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

print(X.shape, y.shape)

(2039, 80, 5) (2039, 1)


In [17]:
# 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

# 훈련data 는 전체 데이터의 70퍼센트
X_train = X[0:split]
y_train = y[0:split]

# 테스트data 는 전체 데이터의 30퍼센트
X_test = X[split:]
y_test = y[split:]

# shape를 출력 -> 분할된 데이터 값으로 출력됨
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1427, 80, 5) (1427, 1)
(612, 80, 5) (612, 1)


In [18]:
#LSTM 모델 구축
model = Sequential()

# 1차원 feature map 생성
model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

# LSTM layer
model.add(LSTM(units = 16, activation = 'tanh'))
model.add(Dense(units = 16, activation = 'sigmoid'))
model.add(Dense(units = 5)) # 출력층 -> 출력층의 unit 수를 조절해서 5일치, 10일치, 20일치, 30일치 등등 예측이 가능!
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 80, 32)            832       
                                                                 
 lstm (LSTM)                 (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 5)                 85        
                                                                 
Total params: 4,325
Trainable params: 4,325
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 모델 컴파일
loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

# 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
earlystopping = EarlyStopping(monitor='loss', patience=10)

# 모델 학습 -> epoch은 100번 진행
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

# 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
predictions = model.predict(X_test)

Epoch 1/100
45/45 [==============================] - 2s 15ms/step - loss: 0.0537 - mse: 0.1075
Epoch 2/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0118 - mse: 0.0235
Epoch 3/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0046 - mse: 0.0091
Epoch 4/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0026 - mse: 0.0052
Epoch 5/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0020 - mse: 0.0040
Epoch 6/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0017 - mse: 0.0033
Epoch 7/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0014 - mse: 0.0028
Epoch 8/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0012 - mse: 0.0025
Epoch 9/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0011 - mse: 0.0021
Epoch 10/100
45/45 [==============================] - 1s 11ms/step - loss: 9.3803e-04 - mse: 0.0019
Epoch 11/100
45/45 [=========================

45/45 [==============================] - 1s 12ms/step - loss: 2.5937e-04 - mse: 5.1875e-04
Epoch 82/100
45/45 [==============================] - 1s 12ms/step - loss: 2.6223e-04 - mse: 5.2447e-04
Epoch 83/100
45/45 [==============================] - 1s 12ms/step - loss: 2.4297e-04 - mse: 4.8594e-04
Epoch 84/100
45/45 [==============================] - 1s 12ms/step - loss: 2.4949e-04 - mse: 4.9898e-04
Epoch 85/100
45/45 [==============================] - 1s 12ms/step - loss: 2.5143e-04 - mse: 5.0287e-04
Epoch 86/100
45/45 [==============================] - 1s 12ms/step - loss: 2.3234e-04 - mse: 4.6468e-04
Epoch 87/100
45/45 [==============================] - 1s 12ms/step - loss: 2.4750e-04 - mse: 4.9501e-04
Epoch 88/100
45/45 [==============================] - 1s 12ms/step - loss: 2.3273e-04 - mse: 4.6546e-04
Epoch 89/100
45/45 [==============================] - 1s 12ms/step - loss: 2.2947e-04 - mse: 4.5894e-04
Epoch 90/100
45/45 [==============================] - 1s 12ms/step - loss: 2.

In [20]:
#예측 비교하기
print(predictions)

[[0.899336   0.90503806 0.90403134 0.90041906 0.90595776]
 [0.89602554 0.90318865 0.9015234  0.8986266  0.9032716 ]
 [0.8941943  0.9019765  0.89986306 0.89731663 0.90194947]
 ...
 [0.24902022 0.34137678 0.33054683 0.33250248 0.34742382]
 [0.24464618 0.33551252 0.32504752 0.328148   0.34101096]
 [0.23446043 0.32494152 0.31487015 0.31858733 0.3295593 ]]


In [21]:
print(y_test) #predictions 와 대충 값이 일치해야함

[[0.8735119 ]
 [0.87103175]
 [0.8859127 ]
 [0.83878968]
 [0.88839286]
 [0.84871032]
 [0.85863095]
 [0.86359127]
 [0.83878968]
 [0.83878968]
 [0.81646825]
 [0.82142857]
 [0.8015873 ]
 [0.80654762]
 [0.82638889]
 [0.82390873]
 [0.83382937]
 [0.82390873]
 [0.83382937]
 [0.81150794]
 [0.80902778]
 [0.79662698]
 [0.76934524]
 [0.75446429]
 [0.74206349]
 [0.74206349]
 [0.76190476]
 [0.75446429]
 [0.7594246 ]
 [0.83134921]
 [0.83382937]
 [0.8139881 ]
 [0.8139881 ]
 [0.82142857]
 [0.80654762]
 [0.79414683]
 [0.8139881 ]
 [0.82638889]
 [0.81646825]
 [0.82142857]
 [0.8139881 ]
 [0.80902778]
 [0.79166667]
 [0.79414683]
 [0.76934524]
 [0.7718254 ]
 [0.76438492]
 [0.7718254 ]
 [0.75198413]
 [0.72470238]
 [0.75446429]
 [0.78174603]
 [0.78174603]
 [0.78422619]
 [0.81150794]
 [0.79662698]
 [0.8015873 ]
 [0.81150794]
 [0.80902778]
 [0.79910714]
 [0.81894841]
 [0.78670635]
 [0.8015873 ]
 [0.80406746]
 [0.8015873 ]
 [0.78174603]
 [0.76686508]
 [0.82390873]
 [0.83878968]
 [0.84871032]
 [0.85367063]
 [0.85

In [22]:
# 실제값 변환
# 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
close_min = csv_df['Close'].min()
close_max = csv_df['Close'].max()

print(csv_df['Close'].min())
print(csv_df['Close'].max())

65900
267500


In [23]:
# MinMaxScaler이용해서 실제값으로 역변환
scaler2 = MinMaxScaler()
scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

# MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

# 예측한 출력값을 실제값으로 역변환
original_pred_values = scaler2.inverse_transform(predictions)

# 역변환된 예측값 출력
original_pred_values

array([[247206.12, 248355.67, 248152.72, 247424.48, 248541.08],
       [246538.73, 247982.83, 247647.12, 247063.12, 247999.56],
       [246169.58, 247738.47, 247312.39, 246799.03, 247733.02],
       ...,
       [116102.48, 134721.56, 132538.23, 132932.5 , 135940.64],
       [115220.67, 133539.33, 131429.58, 132054.64, 134647.8 ],
       [113167.22, 131408.2 , 129377.82, 130127.2 , 132339.16]],
      dtype=float32)

In [24]:
# 테스트 data의 실제값 -> 정규화시켰던 값을 split했었기 때문에 원본 데이터에서 다시 분리해서 가져옴

# original_label_df -> 데이터셋의 실제 종가값
original_label_df = csv_df[label_cols]

# 다시 테스트용만큼의 값으로 분리
original_y_train = original_label_df[0:split]
original_y_test = original_label_df[split:]
original_y_test = original_y_test[window_size:] # window_size만큼의 데이터 제거(예측에 사용하였으므로)

# 역변환된 실제값 출력(종가)
original_y_test

,Close
Date,
2021-02-19,242000
2021-02-22,241500
2021-02-23,244500
2021-02-24,235000
2021-02-25,245000
...,...
2023-08-02,192500
2023-08-03,191700
2023-08-04,192200


In [25]:
# 실제로 에측하기
# feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
pred_feature = feature_df.tail(window_size)

pred_feature

,Close,BPS,EPS,금리,환율
Date,,,,,
2023-04-13,0.659722,0.478161,0.520453,1.0,0.689593
2023-04-14,0.645833,0.478161,0.520453,1.0,0.634754
2023-04-17,0.640377,0.478161,0.520453,1.0,0.639751
2023-04-18,0.622520,0.478161,0.520453,1.0,0.672270
2023-04-19,0.633433,0.478161,0.520453,1.0,0.671835
...,...,...,...,...,...
2023-08-02,0.627976,1.000000,0.887636,1.0,0.603029
2023-08-03,0.624008,1.000000,0.887636,1.0,0.619225
2023-08-04,0.626488,1.000000,0.887636,1.0,0.621864


In [26]:
pred_feature_list = []
pred_feature_list.append(pred_feature)
pred_feature = np.array(pred_feature_list)
print(pred_feature.shape)

(1, 80, 5)


In [27]:
# 5일 예측 -> 5일까지의 예측값 5개 출력
predictions_5d = model.predict(pred_feature)
pred_values_5d = scaler2.inverse_transform(predictions_5d)
print(pred_values_5d)
print(len(pred_values_5d[0]))

1/1 [==============================] - 0s 17ms/step
[[112357.78 130274.6  128355.1  129282.94 131167.34]]
5


In [28]:
# 예측값 추가해서 DataFrame으로 만들기
# 2020년 부터 현재까지의 종가값을 제시하고 앞에 5일 예측값을 추가
pred5dCsvDf = csv_df[['Close']].loc['2020-01-01':,:]

for i in range (len(pred_values_5d[0])):
  pred5dCsvDf.loc[(i+1),:]=[pred_values_5d[0,i]]

pred5dCsvDf

,Close
Date,
2020-01-02,118000.000000
2020-01-03,116000.000000
2020-01-06,116000.000000
2020-01-07,115500.000000
2020-01-08,112000.000000
...,...
1,112357.781250
2,130274.601562
3,128355.101562


In [29]:
# DataFrame을 csv파일로 만들기
pred5dCsvDf.to_csv('5d_predict_filename.csv', header=False, index=True)



---



### **10일치 예측**

In [30]:
window_size = 80

X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

print(X.shape, y.shape)

(2039, 80, 5) (2039, 1)


In [31]:
# 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

# 훈련data 는 전체 데이터의 70퍼센트
X_train = X[0:split]
y_train = y[0:split]

# 테스트data 는 전체 데이터의 30퍼센트
X_test = X[split:]
y_test = y[split:]

# shape를 출력 -> 분할된 데이터 값으로 출력됨
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1427, 80, 5) (1427, 1)
(612, 80, 5) (612, 1)


In [32]:
#LSTM 모델 구축
model = Sequential()

# 1차원 feature map 생성
model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

# LSTM layer
model.add(LSTM(units = 16, activation = 'tanh'))
model.add(Dense(units = 16, activation = 'sigmoid'))
model.add(Dense(units = 10)) # 출력층 -> 출력층의 unit 수를 조절해서 5일치, 10일치, 20일치, 30일치 등등 예측이 가능!
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 80, 32)            832       
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 10)                170       
                                                                 
Total params: 4,410
Trainable params: 4,410
Non-trainable params: 0
_________________________________________________________________


In [33]:
# 모델 컴파일
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

# 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
earlystopping = EarlyStopping(monitor='loss', patience=10)

# 모델 학습 -> epoch은 100번 진행
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

# 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
predictions = model.predict(X_test)

Epoch 1/100
45/45 [==============================] - 2s 12ms/step - loss: 0.2821 - mse: 0.6005
Epoch 2/100
45/45 [==============================] - 1s 12ms/step - loss: 0.1167 - mse: 0.2348
Epoch 3/100
45/45 [==============================] - 1s 13ms/step - loss: 0.0460 - mse: 0.0920
Epoch 4/100
45/45 [==============================] - 1s 13ms/step - loss: 0.0219 - mse: 0.0439
Epoch 5/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0128 - mse: 0.0255
Epoch 6/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0095 - mse: 0.0190
Epoch 7/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0084 - mse: 0.0168
Epoch 8/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0080 - mse: 0.0161
Epoch 9/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0079 - mse: 0.0158
Epoch 10/100
45/45 [==============================] - 1s 12ms/step - loss: 0.0079 - mse: 0.0157
Epoch 11/100
45/45 [=============================

45/45 [==============================] - 1s 12ms/step - loss: 2.9858e-04 - mse: 5.9717e-04
Epoch 84/100
45/45 [==============================] - 1s 12ms/step - loss: 2.9973e-04 - mse: 5.9947e-04
Epoch 85/100
45/45 [==============================] - 1s 12ms/step - loss: 2.7908e-04 - mse: 5.5817e-04
Epoch 86/100
45/45 [==============================] - 1s 12ms/step - loss: 3.2411e-04 - mse: 6.4822e-04
Epoch 87/100
45/45 [==============================] - 1s 12ms/step - loss: 3.1308e-04 - mse: 6.2616e-04
Epoch 88/100
45/45 [==============================] - 1s 12ms/step - loss: 2.8149e-04 - mse: 5.6298e-04
Epoch 89/100
45/45 [==============================] - 1s 12ms/step - loss: 3.2517e-04 - mse: 6.5034e-04
Epoch 90/100
45/45 [==============================] - 1s 12ms/step - loss: 3.0845e-04 - mse: 6.1690e-04
Epoch 91/100
45/45 [==============================] - 1s 12ms/step - loss: 2.6766e-04 - mse: 5.3532e-04
Epoch 92/100
45/45 [==============================] - 1s 12ms/step - loss: 2.

In [34]:
#예측 비교하기
print(predictions)

[[0.8959855  0.92618346 0.8964844  ... 0.9263644  0.89932084 0.8812521 ]
 [0.8957199  0.9261332  0.89620847 ... 0.9262908  0.8989005  0.8808384 ]
 [0.8956835  0.9260905  0.896199   ... 0.92623353 0.8989078  0.8807878 ]
 ...
 [0.43251076 0.5314777  0.49516654 ... 0.54579455 0.42914575 0.48633993]
 [0.42423746 0.5223916  0.48636326 ... 0.53679574 0.4216173  0.47923738]
 [0.41410336 0.5100777  0.47558257 ... 0.52441263 0.40949452 0.46618265]]


In [35]:
print(y_test) #predictions 와 대충 값이 일치해야함

[[0.8735119 ]
 [0.87103175]
 [0.8859127 ]
 [0.83878968]
 [0.88839286]
 [0.84871032]
 [0.85863095]
 [0.86359127]
 [0.83878968]
 [0.83878968]
 [0.81646825]
 [0.82142857]
 [0.8015873 ]
 [0.80654762]
 [0.82638889]
 [0.82390873]
 [0.83382937]
 [0.82390873]
 [0.83382937]
 [0.81150794]
 [0.80902778]
 [0.79662698]
 [0.76934524]
 [0.75446429]
 [0.74206349]
 [0.74206349]
 [0.76190476]
 [0.75446429]
 [0.7594246 ]
 [0.83134921]
 [0.83382937]
 [0.8139881 ]
 [0.8139881 ]
 [0.82142857]
 [0.80654762]
 [0.79414683]
 [0.8139881 ]
 [0.82638889]
 [0.81646825]
 [0.82142857]
 [0.8139881 ]
 [0.80902778]
 [0.79166667]
 [0.79414683]
 [0.76934524]
 [0.7718254 ]
 [0.76438492]
 [0.7718254 ]
 [0.75198413]
 [0.72470238]
 [0.75446429]
 [0.78174603]
 [0.78174603]
 [0.78422619]
 [0.81150794]
 [0.79662698]
 [0.8015873 ]
 [0.81150794]
 [0.80902778]
 [0.79910714]
 [0.81894841]
 [0.78670635]
 [0.8015873 ]
 [0.80406746]
 [0.8015873 ]
 [0.78174603]
 [0.76686508]
 [0.82390873]
 [0.83878968]
 [0.84871032]
 [0.85367063]
 [0.85

In [36]:
# MinMaxScaler이용해서 실제값으로 역변환
scaler2 = MinMaxScaler()
scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

# MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

# 예측한 출력값을 실제값으로 역변환
original_pred_values = scaler2.inverse_transform(predictions)

# 역변환된 예측값 출력
original_pred_values

array([[246530.66, 252618.58, 246631.23, ..., 252655.06, 247203.08,
        243560.42],
       [246477.12, 252608.45, 246575.62, ..., 252640.23, 247118.33,
        243477.02],
       [246469.78, 252599.83, 246573.7 , ..., 252628.67, 247119.8 ,
        243466.81],
       ...,
       [153094.17, 173045.9 , 165725.58, ..., 175932.19, 152415.78,
        163946.12],
       [151426.27, 171214.16, 163950.83, ..., 174118.02, 150898.05,
        162514.25],
       [149383.23, 168731.67, 161777.44, ..., 171621.6 , 148454.1 ,
        159882.42]], dtype=float32)

In [37]:
# 테스트 data의 실제값 -> 정규화시켰던 값을 split했었기 때문에 원본 데이터에서 다시 분리해서 가져옴

# original_label_df -> 데이터셋의 실제 종가값
original_label_df = csv_df[label_cols]

# 다시 테스트용만큼의 값으로 분리
original_y_train = original_label_df[0:split]
original_y_test = original_label_df[split:]
original_y_test = original_y_test[window_size:] # window_size만큼의 데이터 제거(예측에 사용하였으므로)

# 역변환된 실제값 출력(종가)
original_y_test

,Close
Date,
2021-02-19,242000
2021-02-22,241500
2021-02-23,244500
2021-02-24,235000
2021-02-25,245000
...,...
2023-08-02,192500
2023-08-03,191700
2023-08-04,192200


In [38]:
# 실제로 에측하기
# feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
pred_feature = feature_df.tail(window_size)

pred_feature

,Close,BPS,EPS,금리,환율
Date,,,,,
2023-04-13,0.659722,0.478161,0.520453,1.0,0.689593
2023-04-14,0.645833,0.478161,0.520453,1.0,0.634754
2023-04-17,0.640377,0.478161,0.520453,1.0,0.639751
2023-04-18,0.622520,0.478161,0.520453,1.0,0.672270
2023-04-19,0.633433,0.478161,0.520453,1.0,0.671835
...,...,...,...,...,...
2023-08-02,0.627976,1.000000,0.887636,1.0,0.603029
2023-08-03,0.624008,1.000000,0.887636,1.0,0.619225
2023-08-04,0.626488,1.000000,0.887636,1.0,0.621864


In [39]:
pred_feature_list = []
pred_feature_list.append(pred_feature)
pred_feature = np.array(pred_feature_list)
print(pred_feature.shape)

(1, 80, 5)


In [40]:
# 10일 예측 -> 10일까지의 예측값 10개 출력
predictions_10d = model.predict(pred_feature)
pred_values_10d = scaler2.inverse_transform(predictions_10d)
print(pred_values_10d)
print(len(pred_values_10d[0]))

1/1 [==============================] - 0s 16ms/step
[[148289.03 167201.48 160610.33 169921.83 182194.8  157614.73 163711.19
  170157.36 147217.11 158436.64]]
10


In [41]:
# 예측값 추가해서 CSV파일로 만들기
# 2020년 부터 현재까지의 종가값을 제시하고 앞에 10일 예측값을 추가
pred10dCsvDf = csv_df[['Close']].loc['2020-01-01':,:]

for i in range (len(pred_values_10d[0])):
  pred10dCsvDf.loc[(i+1),:]=[pred_values_10d[0,i]]

pred10dCsvDf

,Close
Date,
2020-01-02,118000.000000
2020-01-03,116000.000000
2020-01-06,116000.000000
2020-01-07,115500.000000
2020-01-08,112000.000000
...,...
6,157614.734375
7,163711.187500
8,170157.359375


In [42]:
pred10dCsvDf.to_csv('10d_predict_filename.csv', header=False, index=True)



---



In [43]:
#현재 종가값만 따로 csv파일로 저장

predNowCsvDf = csv_df[['Close']].loc['2020-01-01':,:]

predNowCsvDf.to_csv('Data_filename.csv', header=False, index=True)

In [57]:
day_five = pred_values_5d.flatten().tolist()

In [58]:
day_ten = pred_values_10d.flatten().tolist()

In [62]:
import requests

data = {
    'day_five': day_five,
    'day_ten': day_ten
}

server_url = 'https://616d-39-118-146-59.ngrok-free.app/prediction'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공
